In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from coffea import util
from coffea.processor import accumulate
import os, sys
import re
import pandas as pd
import hist
import matplotlib as mpl
import mplhep
import utils
from pathlib import Path

sys.path.append("../../../analysisTools/")
import plotTools as ptools

In [2]:
def add_signal_info_to_df(df):
    m1_list = []
    delta_list = []
    ctau_list = []
    
    for point in df.index.values:
        sig_dict = ptools.signalPoint(point)
        m1_list.append(sig_dict['m1'])
        delta_list.append(sig_dict['delta'])
        ctau_list.append(sig_dict['ctau'])
    
    df['m1'] = m1_list
    df['delta'] = delta_list
    df['ctau'] = ctau_list
    
    df = df.sort_values(by=['m1']) # sort by m1

    return df

In [3]:
def save_df_to_csv(df, outdir, outname, isSignal = False):
    Path(outdir).mkdir(parents=True, exist_ok=True)

    if isSignal:
        df = add_signal_info_to_df(df)
    
    df.to_csv(f'{outdir}/{outname}.csv')

    print(f'Saved: {outdir}/{outname}.csv')

## All Lxy Bin

In [4]:
prefix = './coffea/skimmed/'
suffix = '_ctau-10-BDT-based_AllLxy'

In [5]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [6]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000000
DY,0.000011,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000000
Triboson,0.001933,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000000
W+jets,0.002310,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000000
Z+jets,0.006645,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_ctau-10-BDT-based_Background_Efficiency_ctau-10_AllLxyBin'

df_to_save = bkg_eff_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/Efficiency//SR_ctau-10-BDT-based_Background_Efficiency_ctau-10_AllLxyBin.csv


In [8]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
QCD,"38,039.83","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,0.00
Top,"5,667.49",559.22,349.76,314.03,312.99,312.95,294.00,0.29
DY,"14,495.72",671.44,623.25,623.25,623.25,623.25,455.65,0.00
Diboson,"4,424.20",307.91,227.24,213.36,211.27,210.33,189.35,0.58
Triboson,53.00,4.32,2.85,2.51,2.51,2.50,2.09,0.00
W+jets,"227,212.60","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39",10.48
Z+jets,"142,220.15","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76",2.66
Total,"432,113.00","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50",14.02


In [9]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_ctau-10-BDT-based_Background_EventCount_Weighted_ctau-10_AllLxyBin'

df_to_save = bkg_cts_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_ctau-10-BDT-based_Background_EventCount_Weighted_ctau-10_AllLxyBin.csv


In [10]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
QCD,12311,381,160,159,159,158,3,0
Top,49818,4688,3058,2348,2337,2336,2006,2
DY,205,13,11,11,11,11,9,0
Diboson,18067,1234,915,860,852,848,767,2
Triboson,2257,170,116,106,105,104,92,0
W+jets,697657,48425,37015,35472,35415,35377,29090,38
Z+jets,1281459,67427,52627,50986,50934,50903,44480,28
Total,2061774,122338,93902,89942,89813,89737,76447,70


In [11]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_ctau-10-BDT-based_Background_EventCount_Raw_ctau-10_AllLxyBin'

df_to_save = bkg_cts_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Raw//SR_ctau-10-BDT-based_Background_EventCount_Raw_ctau-10_AllLxyBin.csv


In [12]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 14.02


In [13]:
prefix = './coffea/skimmed/'
suffix = '_skimmed'

In [14]:
# Signal
sig_histo = util.load(f'{prefix}/ctau-10-BDT-based_AllLxy.coffea')[0]

In [15]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.5,1.0,10.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-10
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,55.0,10.0,10.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-10
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.2,0.5,10.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-10
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,52.5,5.0,10.0,50.0,55.0,0.1,sig_2018_Mchi-52p5_dMchi-5p0_ctau-10
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,33.0,6.0,10.0,30.0,36.0,0.2,sig_2018_Mchi-33p0_dMchi-6p0_ctau-10
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,31.5,3.0,10.0,30.0,33.0,0.1,sig_2018_Mchi-31p5_dMchi-3p0_ctau-10
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,44.0,8.0,10.0,40.0,48.0,0.2,sig_2018_Mchi-44p0_dMchi-8p0_ctau-10
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,42.0,4.0,10.0,40.0,44.0,0.1,sig_2018_Mchi-42p0_dMchi-4p0_ctau-10
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,21.0,2.0,10.0,20.0,22.0,0.1,sig_2018_Mchi-21p0_dMchi-2p0_ctau-10
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.5,1.0,10.0,10.0,11.0,0.1,sig_2018_Mchi-10p5_dMchi-1p0_ctau-10


In [16]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'BDT v2 Loose WP']

### Efficiency

In [17]:
pd.options.display.float_format = '{:,.6f}'.format

In [18]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [19]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [20]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,0.010930,0.001187,0.000987,0.000935,0.000931,0.000931,0.000876,0.000273
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,0.031125,0.005252,0.004562,0.004329,0.004329,0.004329,0.004260,0.002147
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,0.010676,0.001022,0.000847,0.000792,0.000792,0.000792,0.000771,0.000190
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,0.032445,0.004564,0.003903,0.003752,0.003752,0.003752,0.003708,0.001537
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,0.019309,0.002550,0.002202,0.002122,0.002120,0.002120,0.002075,0.001035
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,0.020128,0.002757,0.002373,0.002303,0.002301,0.002301,0.002280,0.000912
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,0.024702,0.003809,0.003226,0.003090,0.003090,0.003090,0.003027,0.001441
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,0.025875,0.003520,0.003022,0.002902,0.002902,0.002902,0.002875,0.001201
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,0.016286,0.001989,0.001734,0.001669,0.001669,0.001669,0.001649,0.000570
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,0.012434,0.001380,0.001180,0.001116,0.001116,0.001116,0.001085,0.000348


In [21]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_ctau-10-BDT-based_Signal_ctau-10_samples_Efficiency_ctau-10_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_ctau-10-BDT-based_Signal_ctau-10_samples_Efficiency_ctau-10_AllLxyBin.csv


### Weighted Event count

In [22]:
pd.options.display.float_format = '{:,.2f}'.format

In [23]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [24]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [25]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,19.10,18.10,18.03,18.03,16.97,5.28
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,18.96,3.20,2.78,2.64,2.64,2.64,2.60,1.31
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,"6,090.24",583.27,483.38,451.73,451.73,451.73,439.55,108.37
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,625.95,88.06,75.30,72.38,72.38,72.38,71.54,29.66
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,"15,555.45","2,054.32","1,773.84","1,709.70","1,707.69","1,707.69","1,671.59",833.87
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,"27,263.50","3,734.67","3,214.54","3,119.89","3,117.26","3,117.26","3,088.33","1,234.85"
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,47.30,7.30,6.18,5.92,5.92,5.92,5.80,2.76
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,"1,569.13",213.48,183.29,176.01,176.01,176.01,174.34,72.83
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,"3,043.72",371.80,324.16,312.00,312.00,312.00,308.26,106.51
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,"3,591.89",398.72,340.80,322.41,322.41,322.41,313.43,100.57


In [26]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_ctau-10-BDT-based_Signal_ctau-10_samples_EventCount_Weighted_ctau-10_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_ctau-10-BDT-based_Signal_ctau-10_samples_EventCount_Weighted_ctau-10_AllLxyBin.csv


In [27]:
# without m1 = 30 GeV
ct = 10.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,19.10,18.10,18.03,18.03,16.97,5.28
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,18.96,3.20,2.78,2.64,2.64,2.64,2.60,1.31
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,"6,090.24",583.27,483.38,451.73,451.73,451.73,439.55,108.37
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,625.95,88.06,75.30,72.38,72.38,72.38,71.54,29.66
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,47.30,7.30,6.18,5.92,5.92,5.92,5.80,2.76
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,"1,569.13",213.48,183.29,176.01,176.01,176.01,174.34,72.83
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,"3,043.72",371.80,324.16,312.00,312.00,312.00,308.26,106.51
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,"3,591.89",398.72,340.80,322.41,322.41,322.41,313.43,100.57
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,118.48,13.27,10.81,10.19,10.19,10.19,9.84,3.65
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,94.00,11.49,10.00,9.40,9.40,9.40,9.20,4.22


### Raw count

In [28]:
pd.options.display.float_format = '{:,.0f}'.format

In [29]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [30]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [31]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,3168,344,286,271,270,270,254,79
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,3609,609,529,502,502,502,494,249
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5002,479,397,371,371,371,361,89
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,13492,1898,1623,1560,1560,1560,1542,639
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,7762,1025,885,853,852,852,834,416
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,10374,1421,1223,1187,1186,1186,1175,470
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,9065,1398,1184,1134,1134,1134,1111,529
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,10341,1407,1208,1160,1160,1160,1149,480
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,6515,796,694,668,668,668,660,228
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,7999,888,759,718,718,718,698,224


In [32]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_ctau-10-BDT-based_Signal_ctau-10_samples_EventCount_Raw_ctau-10_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_ctau-10-BDT-based_Signal_ctau-10_samples_EventCount_Raw_ctau-10_AllLxyBin.csv


In [33]:
nBkg

14.018958800901501

In [34]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [35]:
pd.options.display.float_format = '{:,.4f}'.format

In [36]:
df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [37]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.2797,5.0000,0.2000,10.0000,14.0190
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,108.3725,5.0000,0.1000,10.0000,14.0190
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,100.5699,10.0000,0.1000,10.0000,14.0190
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,3.6497,10.0000,0.2000,10.0000,14.0190
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,106.5086,20.0000,0.1000,10.0000,14.0190
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,4.2230,20.0000,0.2000,10.0000,14.0190
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,833.8735,30.0000,0.2000,10.0000,14.0190
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,"1,234.8489",30.0000,0.1000,10.0000,14.0190
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,2.7603,40.0000,0.2000,10.0000,14.0190
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,72.8270,40.0000,0.1000,10.0000,14.0190


In [38]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0000,0.2000,10.0000,5.2797,14.0190,1.4101
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0000,0.1000,10.0000,108.3725,14.0190,28.9442
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.0000,0.1000,10.0000,100.5699,14.0190,26.8603
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,10.0000,0.2000,10.0000,3.6497,14.0190,0.9748
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,20.0000,0.1000,10.0000,106.5086,14.0190,28.4464
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,20.0000,0.2000,10.0000,4.2230,14.0190,1.1279
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,30.0000,0.2000,10.0000,833.8735,14.0190,222.7113
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,30.0000,0.1000,10.0000,"1,234.8489",14.0190,329.8040
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,40.0000,0.2000,10.0000,2.7603,14.0190,0.7372
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,40.0000,0.1000,10.0000,72.8270,14.0190,19.4507


In [39]:
outdir = './csv/SR_Yields/'
outname = 'SR_ctau-10-BDT-based_ctau-10_AllLxy_S_over_B.csv'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/SR_Yields//SR_ctau-10-BDT-based_ctau-10_AllLxy_S_over_B.csv.csv


### ctau = 10mm signal points

In [40]:
# S/sqrt(B) in ctau = 10mm, delta = 0.1
ct = 10.0
delta = 0.1

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0000,0.1000,10.0000,108.3725,14.0190,28.9442
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.0000,0.1000,10.0000,100.5699,14.0190,26.8603
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,20.0000,0.1000,10.0000,106.5086,14.0190,28.4464
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,30.0000,0.1000,10.0000,"1,234.8489",14.0190,329.8040
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,40.0000,0.1000,10.0000,72.8270,14.0190,19.4507
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,50.0000,0.1000,10.0000,29.6551,14.0190,7.9203


In [41]:
# S/sqrt(B) in ctau = 10mm, delta = 0.2
ct = 10.0
delta = 0.2

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0000,0.2000,10.0000,5.2797,14.0190,1.4101
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,10.0000,0.2000,10.0000,3.6497,14.0190,0.9748
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,20.0000,0.2000,10.0000,4.2230,14.0190,1.1279
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,30.0000,0.2000,10.0000,833.8735,14.0190,222.7113
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,40.0000,0.2000,10.0000,2.7603,14.0190,0.7372
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,50.0000,0.2000,10.0000,1.3084,14.0190,0.3494


# SR BDT-ctau-10: High Lxy (Lxy > 0.5)

## Background

In [42]:
prefix = './coffea/skimmed/'
suffix = '_ctau-10-BDT-based_HighLxy'

In [43]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [44]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000003,0.000000
DY,0.000011,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000015,0.000000
Triboson,0.001933,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000034,0.000000
W+jets,0.002310,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000066,0.000000
Z+jets,0.006645,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000173,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [45]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
QCD,"38,039.83","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,7.84,0.00
Top,"5,667.49",559.22,349.76,314.03,312.99,312.95,294.00,139.54,0.00
DY,"14,495.72",671.44,623.25,623.25,623.25,623.25,455.65,48.18,0.00
Diboson,"4,424.20",307.91,227.24,213.36,211.27,210.33,189.35,103.12,0.58
Triboson,53.00,4.32,2.85,2.51,2.51,2.50,2.09,0.93,0.00
W+jets,"227,212.60","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39","6,454.21",7.73
Z+jets,"142,220.15","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76","3,694.44",1.76
Total,"432,113.00","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50","10,448.27",10.07


In [46]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
QCD,12311,381,160,159,159,158,3,1,0
Top,49818,4688,3058,2348,2337,2336,2006,972,0
DY,205,13,11,11,11,11,9,2,0
Diboson,18067,1234,915,860,852,848,767,419,2
Triboson,2257,170,116,106,105,104,92,50,0
W+jets,697657,48425,37015,35472,35415,35377,29090,16594,30
Z+jets,1281459,67427,52627,50986,50934,50903,44480,26108,18
Total,2061774,122338,93902,89942,89813,89737,76447,44146,50


In [47]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 10.07


## Signal

In [48]:
prefix = './coffea/skimmed/'
suffix = '_skimmed'

In [49]:
# Signal
sig_histo = util.load(f'{prefix}/ctau-10-BDT-based_HighLxy.coffea')[0]

In [50]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.5,1.0,10.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-10
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,55.0,10.0,10.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-10
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.2,0.5,10.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-10
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,52.5,5.0,10.0,50.0,55.0,0.1,sig_2018_Mchi-52p5_dMchi-5p0_ctau-10
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,42.0,4.0,10.0,40.0,44.0,0.1,sig_2018_Mchi-42p0_dMchi-4p0_ctau-10
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,44.0,8.0,10.0,40.0,48.0,0.2,sig_2018_Mchi-44p0_dMchi-8p0_ctau-10
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,33.0,6.0,10.0,30.0,36.0,0.2,sig_2018_Mchi-33p0_dMchi-6p0_ctau-10
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,31.5,3.0,10.0,30.0,33.0,0.1,sig_2018_Mchi-31p5_dMchi-3p0_ctau-10
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,21.0,2.0,10.0,20.0,22.0,0.1,sig_2018_Mchi-21p0_dMchi-2p0_ctau-10
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.5,1.0,10.0,10.0,11.0,0.1,sig_2018_Mchi-10p5_dMchi-1p0_ctau-10


In [51]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'Lxy > 0.5',
 'BDT v2 Loose WP']

### Efficiency

In [52]:
pd.options.display.float_format = '{:,.6f}'.format

In [53]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [54]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [55]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,0.010930,0.001187,0.000987,0.000935,0.000931,0.000931,0.000876,0.000638,0.000266
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,0.031125,0.005252,0.004562,0.004329,0.004329,0.004329,0.004260,0.002776,0.001440
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,0.010676,0.001022,0.000847,0.000792,0.000792,0.000792,0.000771,0.000553,0.000184
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,0.032445,0.004564,0.003903,0.003752,0.003752,0.003752,0.003708,0.002436,0.001133
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,0.025875,0.003520,0.003022,0.002902,0.002902,0.002902,0.002875,0.001954,0.000928
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,0.024702,0.003809,0.003226,0.003090,0.003090,0.003090,0.003027,0.002022,0.001027
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,0.019309,0.002550,0.002202,0.002122,0.002120,0.002120,0.002075,0.001428,0.000752
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,0.020128,0.002757,0.002373,0.002303,0.002301,0.002301,0.002280,0.001626,0.000762
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,0.016286,0.001989,0.001734,0.001669,0.001669,0.001669,0.001649,0.001130,0.000472
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,0.012434,0.001380,0.001180,0.001116,0.001116,0.001116,0.001085,0.000802,0.000331


### Weighted Event count

In [56]:
pd.options.display.float_format = '{:,.2f}'.format

In [57]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [58]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [59]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,19.10,18.10,18.03,18.03,16.97,12.36,5.15
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,18.96,3.20,2.78,2.64,2.64,2.64,2.60,1.69,0.88
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,"6,090.24",583.27,483.38,451.73,451.73,451.73,439.55,315.38,104.72
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,625.95,88.06,75.30,72.38,72.38,72.38,71.54,47.00,21.86
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,"1,569.13",213.48,183.29,176.01,176.01,176.01,174.34,118.50,56.28
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,47.30,7.30,6.18,5.92,5.92,5.92,5.80,3.87,1.97
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,"15,555.45","2,054.32","1,773.84","1,709.70","1,707.69","1,707.69","1,671.59","1,150.54",605.43
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,"27,263.50","3,734.67","3,214.54","3,119.89","3,117.26","3,117.26","3,088.33","2,202.30","1,032.46"
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,"3,043.72",371.80,324.16,312.00,312.00,312.00,308.26,211.13,88.30
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,"3,591.89",398.72,340.80,322.41,322.41,322.41,313.43,231.69,95.63


In [60]:
# without m1 = 30 GeV
ct = 10.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,19.10,18.10,18.03,18.03,16.97,12.36,5.15
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,18.96,3.20,2.78,2.64,2.64,2.64,2.60,1.69,0.88
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,"6,090.24",583.27,483.38,451.73,451.73,451.73,439.55,315.38,104.72
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,625.95,88.06,75.30,72.38,72.38,72.38,71.54,47.00,21.86
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,"1,569.13",213.48,183.29,176.01,176.01,176.01,174.34,118.50,56.28
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,47.30,7.30,6.18,5.92,5.92,5.92,5.80,3.87,1.97
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,"3,043.72",371.80,324.16,312.00,312.00,312.00,308.26,211.13,88.30
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,"3,591.89",398.72,340.80,322.41,322.41,322.41,313.43,231.69,95.63
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,94.00,11.49,10.00,9.40,9.40,9.40,9.20,6.75,3.54
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,118.48,13.27,10.81,10.19,10.19,10.19,9.84,7.20,3.25


### Raw count

In [61]:
pd.options.display.float_format = '{:,.0f}'.format

In [62]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [63]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [64]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,3168,344,286,271,270,270,254,185,77
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,3609,609,529,502,502,502,494,322,167
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5002,479,397,371,371,371,361,259,86
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,13492,1898,1623,1560,1560,1560,1542,1013,471
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,10341,1407,1208,1160,1160,1160,1149,781,371
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,9065,1398,1184,1134,1134,1134,1111,742,377
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,7762,1025,885,853,852,852,834,574,302
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,10374,1421,1223,1187,1186,1186,1175,838,393
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,6515,796,694,668,668,668,660,452,189
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,7999,888,759,718,718,718,698,516,213


In [65]:
nBkg

10.066154665178592

In [66]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [67]:
pd.options.display.float_format = '{:,.4f}'.format

df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [68]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.1460,5.0000,0.2000,10.0000,10.0662
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,104.7199,5.0000,0.1000,10.0000,10.0662
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,95.6294,10.0000,0.1000,10.0000,10.0662
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,3.2462,10.0000,0.2000,10.0000,10.0662
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,88.2967,20.0000,0.1000,10.0000,10.0662
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,3.5418,20.0000,0.2000,10.0000,10.0662
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,605.4298,30.0000,0.2000,10.0000,10.0662
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,"1,032.4643",30.0000,0.1000,10.0000,10.0662
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,56.2791,40.0000,0.1000,10.0000,10.0662
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,1.9673,40.0000,0.2000,10.0000,10.0662


In [69]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0000,0.2000,10.0000,5.1460,10.0662,1.6219
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0000,0.1000,10.0000,104.7199,10.0662,33.0063
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.0000,0.1000,10.0000,95.6294,10.0662,30.1411
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,10.0000,0.2000,10.0000,3.2462,10.0662,1.0232
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,20.0000,0.1000,10.0000,88.2967,10.0662,27.8300
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,20.0000,0.2000,10.0000,3.5418,10.0662,1.1163
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,30.0000,0.2000,10.0000,605.4298,10.0662,190.8236
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,30.0000,0.1000,10.0000,"1,032.4643",10.0662,325.4193
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,40.0000,0.1000,10.0000,56.2791,10.0662,17.7384
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,40.0000,0.2000,10.0000,1.9673,10.0662,0.6201


# SR BDT-ctau-10: Low Lxy (Lxy <= 0.5)

## Background

In [70]:
prefix = './coffea/skimmed/'
suffix = '_ctau-10-BDT-based_LowLxy'

In [71]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [72]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000004,0.000000
DY,0.000011,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000012,0.000000
Triboson,0.001933,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000042,0.000000
W+jets,0.002310,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000048,0.000000
Z+jets,0.006645,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000120,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [73]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
QCD,"38,039.83","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,624.41,0.00
Top,"5,667.49",559.22,349.76,314.03,312.99,312.95,294.00,154.46,0.29
DY,"14,495.72",671.44,623.25,623.25,623.25,623.25,455.65,407.47,0.00
Diboson,"4,424.20",307.91,227.24,213.36,211.27,210.33,189.35,86.23,0.00
Triboson,53.00,4.32,2.85,2.51,2.51,2.50,2.09,1.16,0.00
W+jets,"227,212.60","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39","4,750.18",2.76
Z+jets,"142,220.15","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76","2,562.32",0.91
Total,"432,113.00","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50","8,586.23",3.95


In [74]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
QCD,12311,381,160,159,159,158,3,2,0
Top,49818,4688,3058,2348,2337,2336,2006,1034,2
DY,205,13,11,11,11,11,9,7,0
Diboson,18067,1234,915,860,852,848,767,348,0
Triboson,2257,170,116,106,105,104,92,42,0
W+jets,697657,48425,37015,35472,35415,35377,29090,12496,8
Z+jets,1281459,67427,52627,50986,50934,50903,44480,18372,10
Total,2061774,122338,93902,89942,89813,89737,76447,32301,20


In [75]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 3.95


## Signal

In [76]:
prefix = './coffea/skimmed/'
suffix = '_skimmed'

In [77]:
# Signal
sig_histo = util.load(f'{prefix}/ctau-10-BDT-based_LowLxy.coffea')[0]

In [78]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,55.0,10.0,10.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-10
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.5,1.0,10.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-10
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.2,0.5,10.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-10
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,52.5,5.0,10.0,50.0,55.0,0.1,sig_2018_Mchi-52p5_dMchi-5p0_ctau-10
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,44.0,8.0,10.0,40.0,48.0,0.2,sig_2018_Mchi-44p0_dMchi-8p0_ctau-10
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,42.0,4.0,10.0,40.0,44.0,0.1,sig_2018_Mchi-42p0_dMchi-4p0_ctau-10
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,31.5,3.0,10.0,30.0,33.0,0.1,sig_2018_Mchi-31p5_dMchi-3p0_ctau-10
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,33.0,6.0,10.0,30.0,36.0,0.2,sig_2018_Mchi-33p0_dMchi-6p0_ctau-10
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,21.0,2.0,10.0,20.0,22.0,0.1,sig_2018_Mchi-21p0_dMchi-2p0_ctau-10
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.5,1.0,10.0,10.0,11.0,0.1,sig_2018_Mchi-10p5_dMchi-1p0_ctau-10


In [79]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'Lxy <= 0.5',
 'BDT v2 Loose WP']

### Efficiency

In [80]:
pd.options.display.float_format = '{:,.6f}'.format

In [81]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [82]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [83]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,0.031125,0.005252,0.004562,0.004329,0.004329,0.004329,0.004260,0.001484,0.000707
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,0.010930,0.001187,0.000987,0.000935,0.000931,0.000931,0.000876,0.000238,0.000007
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,0.010676,0.001022,0.000847,0.000792,0.000792,0.000792,0.000771,0.000218,0.000006
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,0.032445,0.004564,0.003903,0.003752,0.003752,0.003752,0.003708,0.001272,0.000404
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,0.024702,0.003809,0.003226,0.003090,0.003090,0.003090,0.003027,0.001005,0.000414
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,0.025875,0.003520,0.003022,0.002902,0.002902,0.002902,0.002875,0.000921,0.000273
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,0.020128,0.002757,0.002373,0.002303,0.002301,0.002301,0.002280,0.000654,0.000149
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,0.019309,0.002550,0.002202,0.002122,0.002120,0.002120,0.002075,0.000647,0.000284
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,0.016286,0.001989,0.001734,0.001669,0.001669,0.001669,0.001649,0.000520,0.000097
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,0.012434,0.001380,0.001180,0.001116,0.001116,0.001116,0.001085,0.000283,0.000017


### Weighted Event count

In [84]:
pd.options.display.float_format = '{:,.2f}'.format

In [85]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [86]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [87]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,18.96,3.20,2.78,2.64,2.64,2.64,2.60,0.90,0.43
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,19.10,18.10,18.03,18.03,16.97,4.61,0.13
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,"6,090.24",583.27,483.38,451.73,451.73,451.73,439.55,124.17,3.65
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,625.95,88.06,75.30,72.38,72.38,72.38,71.54,24.54,7.80
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,47.30,7.30,6.18,5.92,5.92,5.92,5.80,1.93,0.79
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,"1,569.13",213.48,183.29,176.01,176.01,176.01,174.34,55.84,16.55
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,"27,263.50","3,734.67","3,214.54","3,119.89","3,117.26","3,117.26","3,088.33",886.02,202.38
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,"15,555.45","2,054.32","1,773.84","1,709.70","1,707.69","1,707.69","1,671.59",521.04,228.44
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,"3,043.72",371.80,324.16,312.00,312.00,312.00,308.26,97.13,18.21
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,"3,591.89",398.72,340.80,322.41,322.41,322.41,313.43,81.74,4.94


In [88]:
# without m1 = 30 GeV
ct = 10.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,18.96,3.20,2.78,2.64,2.64,2.64,2.60,0.90,0.43
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,19.10,18.10,18.03,18.03,16.97,4.61,0.13
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,"6,090.24",583.27,483.38,451.73,451.73,451.73,439.55,124.17,3.65
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,625.95,88.06,75.30,72.38,72.38,72.38,71.54,24.54,7.80
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,47.30,7.30,6.18,5.92,5.92,5.92,5.80,1.93,0.79
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,"1,569.13",213.48,183.29,176.01,176.01,176.01,174.34,55.84,16.55
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,"3,043.72",371.80,324.16,312.00,312.00,312.00,308.26,97.13,18.21
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,"3,591.89",398.72,340.80,322.41,322.41,322.41,313.43,81.74,4.94
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,94.00,11.49,10.00,9.40,9.40,9.40,9.20,2.44,0.68
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,118.48,13.27,10.81,10.19,10.19,10.19,9.84,2.64,0.40


### Raw count

In [89]:
pd.options.display.float_format = '{:,.0f}'.format

In [90]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [91]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [92]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,3609,609,529,502,502,502,494,172,82
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,3168,344,286,271,270,270,254,69,2
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5002,479,397,371,371,371,361,102,3
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,13492,1898,1623,1560,1560,1560,1542,529,168
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,9065,1398,1184,1134,1134,1134,1111,369,152
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,10341,1407,1208,1160,1160,1160,1149,368,109
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,10374,1421,1223,1187,1186,1186,1175,337,77
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,7762,1025,885,853,852,852,834,260,114
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,6515,796,694,668,668,668,660,208,39
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,7999,888,759,718,718,718,698,182,11


In [93]:
nBkg

3.95280413572291

In [94]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [95]:
pd.options.display.float_format = '{:,.4f}'.format

df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [96]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,0.1337,5.0000,0.2000,10.0000,3.9528
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,3.6526,5.0000,0.1000,10.0000,3.9528
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,4.9405,10.0000,0.1000,10.0000,3.9528
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,0.4035,10.0000,0.2000,10.0000,3.9528
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,18.2119,20.0000,0.1000,10.0000,3.9528
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,0.6813,20.0000,0.2000,10.0000,3.9528
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,202.3846,30.0000,0.1000,10.0000,3.9528
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,228.4437,30.0000,0.2000,10.0000,3.9528
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,0.7930,40.0000,0.2000,10.0000,3.9528
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,16.5479,40.0000,0.1000,10.0000,3.9528


In [97]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0000,0.2000,10.0000,0.1337,3.9528,0.0673
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0000,0.1000,10.0000,3.6526,3.9528,1.8372
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.0000,0.1000,10.0000,4.9405,3.9528,2.4849
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,10.0000,0.2000,10.0000,0.4035,3.9528,0.2029
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,20.0000,0.1000,10.0000,18.2119,3.9528,9.1602
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,20.0000,0.2000,10.0000,0.6813,3.9528,0.3427
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,30.0000,0.1000,10.0000,202.3846,3.9528,101.7946
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,30.0000,0.2000,10.0000,228.4437,3.9528,114.9017
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,40.0000,0.2000,10.0000,0.7930,3.9528,0.3989
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,40.0000,0.1000,10.0000,16.5479,3.9528,8.3232
